## Imports

In [1]:
# Standard library imports,
import time
import random
import string
import datetime

# Third-party imports,
import numpy as np
import matplotlib.pyplot as plt
from dataset import COVID_DATASET
import shap
import pandas as pd

# PyTorch imports,
import torch
from torchsummary import summary
from torch.optim import SGD
from torch.utils.data import Dataset, DataLoader

##  Model Architecture & Training Method

In [15]:
class ConfigDataset(Dataset):
    def __init__(self, path):
        """X is the training input while Y is the target output."""

        # Loading PyTorch dataset,
        dataset = torch.load(path)
        self.X = dataset.X
        self.Y = dataset.Y
        self.features = dataset.features
        self.classes = dataset.classes
        del dataset

    def __len__(self):
        """Returns the number of objects in the dataset."""
        return self.X.shape[0]

    def __getitem__(self, index):
        "Returns a tuple of training input and target output."
        return self.X[index], self.Y[index]

class NeuralNetwork(torch.nn.Module):
    """The neutral network architecture."""

    def __init__(self):
        """Creating the layer stucture and activation functions of the neutral network."""
        super().__init__()

        # Layers,
        self.DenseLayer_INPUT = torch.nn.Linear(in_features = 48, out_features = 96)
        self.DenseLayer_HIDDEN_1 = torch.nn.Linear(in_features = 96, out_features = 96)
        self.DenseLayer_HIDDEN_2 = torch.nn.Linear(in_features = 96, out_features = 96)
        self.DenseLayer_HIDDEN_3 = torch.nn.Linear(in_features = 96, out_features = 24)
        self.DenseLayer_OUTPUT = torch.nn.Linear(in_features = 24, out_features = 2)

        # Activation functions,
        self.ReLU = torch.nn.ReLU()
        self.SoftMax = torch.nn.Softmax()

    def forward(self, X):
        """Defining the forward propagation."""

        X = self.DenseLayer_INPUT(X)
        X = self.ReLU(self.DenseLayer_HIDDEN_1(X))
        X = self.ReLU(self.DenseLayer_HIDDEN_2(X))
        X = self.ReLU(self.DenseLayer_HIDDEN_3(X))
        X = self.SoftMax(self.DenseLayer_OUTPUT(X))

        return X

def TrainModel(training_data, neural_network, model_name, n_epochs = 10, learning_rate = 0.01):

    print(model_name)
    summary(neural_network, (1, 48))
    print(" ")

    # Utilisation of CUDA if possible,
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print(f'CUDA is available. Using GPU: {torch.cuda.get_device_name(0)}')
    else:
        device = torch.device("cpu")
        print('CUDA is not available. Using CPU.')

    neural_network.to(device)

    # Stochastic Gradient Descent (SGD) as optimiser,
    optimiser = SGD(neural_network.parameters(), lr = learning_rate)

    # Cross-Entropy loss function,
    loss_function = torch.nn.CrossEntropyLoss()

    losses = []
    epochs = np.arange(start = 1, stop = n_epochs, step = 1)

    for epoch in range(1, n_epochs + 1):
        epoch_loss = []
        stopwatch_start = time.time()
        for i, (X, Y) in enumerate(training_data):

            # Move data to selected device,
            X, Y = X.to(device), Y.to(device)

            # Back propagation,
            optimiser.zero_grad() 
            loss_value = loss_function(neural_network(X), Y) 
            epoch_loss.append(loss_value.item())
            loss_value.backward()
            optimiser.step()

        stopwatch_stop = time.time()
        epoch_time = round(stopwatch_stop - stopwatch_start, 2)
        epoch_avgloss = np.mean(epoch_loss)
        losses.append(epoch_avgloss)
        ETA = str(datetime.timedelta(seconds = (n_epochs - epoch)*epoch_time)).split(".")[0]
        update_string = f'[Completed Epoch: {epoch}/{n_epochs} - Time Taken: {epoch_time} secs - Loss: {epoch_avgloss:.5f} - ETA: {ETA} ]'
        print(update_string)

    torch.save(neural_network.state_dict(), f'{model_name}.pth')
    return np.array(epochs), np.array(losses)

def AssessModel(test_data, neural_network):

    X_test = test_data[:][0]
    Y_test = test_data[:][1]
    Yhat_test = neural_network(X_test)

    correct_counter = 0
    incorrect_counter = 0
    for Y, Yhat in zip(Y_test, Yhat_test):
        print(Y, Yhat)
        if Yhat.argmax() == Y.argmax():
            correct_counter += 1
        else:
            incorrect_counter += 1

    accuracy = correct_counter/(correct_counter + incorrect_counter)

    return accuracy

## Model Training Algorithm

In [3]:
# Hyperparameters,
EPOCHS = 25
BATCH_SIZE = 8
LEARNING_RATE = 0.01

if __name__ == "__main__":

    ID = ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(5))
    model_name = "Model-" + ID

    # Creating and training neural network,
    training_dataset = ConfigDataset("training.pt")
    training_dataset_loaded = DataLoader(training_dataset, BATCH_SIZE)

    # Print example,
    train_features, train_labels = next(iter(training_dataset_loaded))
    print(f"Input Batch Shape: {train_features.size()}")
    print(f"Target Batch Shape: {train_labels.size()}")
    random_index = random.randint(0, BATCH_SIZE - 1)
    label = train_labels[random_index]
    print(f"Example Label: {label}")

    # Creating and training neural network,
    neural_network = NeuralNetwork()
    epoch_array, loss_array = TrainModel(training_dataset_loaded, neural_network, model_name, EPOCHS, LEARNING_RATE)

    # Testing neural network,
    test_dataset = ConfigDataset("test.pt")
    print("Accuracy: " + str(AssessModel(test_dataset, neural_network)))

Input Batch Shape: torch.Size([8, 48])
Target Batch Shape: torch.Size([8, 2])
Example Label: tensor([0., 1.])
Model-6PMIV
Layer (type:depth-idx)                   Output Shape              Param #
├─Linear: 1-1                            [-1, 1, 96]               4,704
├─Linear: 1-2                            [-1, 1, 96]               9,312
├─ReLU: 1-3                              [-1, 1, 96]               --
├─Linear: 1-4                            [-1, 1, 96]               9,312
├─ReLU: 1-5                              [-1, 1, 96]               --
├─Linear: 1-6                            [-1, 1, 24]               2,328
├─ReLU: 1-7                              [-1, 1, 24]               --
├─Linear: 1-8                            [-1, 1, 2]                50
├─Softmax: 1-9                           [-1, 1, 2]                --
Total params: 25,706
Trainable params: 25,706
Non-trainable params: 0
Total mult-adds (M): 0.03
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Param

Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[Completed Epoch: 1/25 - Time Taken: 12.66 secs - Loss: 0.46164 - ETA: 0:05:03 ]
[Completed Epoch: 2/25 - Time Taken: 13.14 secs - Loss: 0.43000 - ETA: 0:05:02 ]
[Completed Epoch: 3/25 - Time Taken: 16.61 secs - Loss: 0.40742 - ETA: 0:06:05 ]
[Completed Epoch: 4/25 - Time Taken: 15.34 secs - Loss: 0.40586 - ETA: 0:05:22 ]
[Completed Epoch: 5/25 - Time Taken: 14.32 secs - Loss: 0.40484 - ETA: 0:04:46 ]
[Completed Epoch: 6/25 - Time Taken: 13.28 secs - Loss: 0.40414 - ETA: 0:04:12 ]
[Completed Epoch: 7/25 - Time Taken: 13.2 secs - Loss: 0.40372 - ETA: 0:03:57 ]
[Completed Epoch: 8/25 - Time Taken: 13.14 secs - Loss: 0.40324 - ETA: 0:03:43 ]
[Completed Epoch: 9/25 - Time Taken: 13.03 secs - Loss: 0.40279 - ETA: 0:03:28 ]
[Completed Epoch: 10/25 - Time Taken: 13.5 secs - Loss: 0.40243 - ETA: 0:03:22 ]
[Completed Epoch: 11/25 - Time Taken: 14.2 secs - Loss: 0.40231 - ETA: 0:03:18 ]
[Completed Epoch: 12/25 - Time Taken: 14.77 secs - Loss: 0.40212 - ETA: 0:03:12 ]
[Completed Epoch: 13/25 - Ti

## Model Loader

In [18]:
MODEL_FILE = "Model-6PMIV.pth"

# Loading model,
model = NeuralNetwork()
model.load_state_dict(torch.load(MODEL_FILE))
model.eval()

# Passing test data,
test_dataset = ConfigDataset("test.pt")
print("Accuracy: " + str(AssessModel(test_dataset, model)))

Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor([1., 0.]) tensor([1.0000e+00, 1.0550e-06], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 2.7419e-07], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([2.4805e-04, 9.9975e-01], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 1.7842e-07], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([4.1410e-05, 9.9996e-01], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([1.5245e-06, 1.0000e+00], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 5.2917e-09], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 1.5571e-06], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 9.1059e-07], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([0.9961, 0.0039], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([8.2197e-05, 9.9992e-01], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 1.4161e-08], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([1.7915e-05, 9.9998e-01], grad_fn=<UnbindBackward0>)
tensor(

tensor([1., 0.]) tensor([1.0000e+00, 4.1071e-08], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 6.1342e-09], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([0.7650, 0.2350], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([0.2014, 0.7986], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([9.5888e-06, 9.9999e-01], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 5.5456e-07], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([1.9788e-08, 1.0000e+00], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 1.6938e-07], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 1.9266e-08], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 3.6534e-07], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([1.1643e-06, 1.0000e+00], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 3.1686e-08], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([9.9999e-01, 7.6902e-06], grad_fn=<UnbindBackward0>)
tensor([0., 1.]

tensor([1., 0.]) tensor([9.9999e-01, 1.0525e-05], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([0.9779, 0.0221], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([1.7722e-04, 9.9982e-01], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 8.8402e-07], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([1.9760e-04, 9.9980e-01], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([1.0000e+00, 2.7869e-07], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([6.0937e-04, 9.9939e-01], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 1.5312e-06], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 7.3688e-09], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([0.5618, 0.4382], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 9.4628e-08], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 3.7517e-09], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([2.7447e-04, 9.9973e-01], grad_fn=<UnbindBackward0>)
tensor([1., 0.]

tensor([0., 1.]) tensor([0.0064, 0.9936], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 1.9791e-06], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.9851e-05, 9.9998e-01], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 6.1342e-09], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([3.3101e-05, 9.9997e-01], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([1.3331e-05, 9.9999e-01], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([6.3586e-04, 9.9936e-01], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([5.7580e-04, 9.9942e-01], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([9.9994e-01, 5.6029e-05], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([1.1645e-05, 9.9999e-01], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([3.2116e-05, 9.9997e-01], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([9.9995e-01, 4.6892e-05], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([2.0616e-07, 1.0000e+00], grad_fn=<UnbindBackward0>)
tensor(

...

tensor([1., 0.]) tensor([1.0000e+00, 3.8801e-07], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([0.0335, 0.9665], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([9.3716e-06, 9.9999e-01], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 8.6676e-08], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([1.0164e-06, 1.0000e+00], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 4.6955e-09], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 6.0941e-07], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([6.2338e-07, 1.0000e+00], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([4.8539e-08, 1.0000e+00], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([7.2143e-08, 1.0000e+00], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([8.8266e-04, 9.9912e-01], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 1.5480e-08], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 1.9051e-08], grad_fn=<UnbindBackward0>)
tensor(

tensor([0., 1.]) tensor([0.0353, 0.9647], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([4.8116e-05, 9.9995e-01], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([0.2164, 0.7836], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 1.5255e-06], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([1.2911e-04, 9.9987e-01], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([0.0150, 0.9850], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 3.1903e-07], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 2.4678e-08], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([2.9740e-04, 9.9970e-01], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 3.2965e-08], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 5.1907e-07], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([2.8929e-05, 9.9997e-01], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([4.0872e-06, 1.0000e+00], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor

tensor([0., 1.]) tensor([1.5590e-07, 1.0000e+00], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([6.7093e-06, 9.9999e-01], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 7.5368e-08], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([6.1961e-07, 1.0000e+00], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([0.0061, 0.9939], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([0.0926, 0.9074], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([0.9792, 0.0208], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 4.6123e-06], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 1.3600e-07], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([3.6997e-06, 1.0000e+00], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([2.3495e-04, 9.9977e-01], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([9.9421e-06, 9.9999e-01], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([0.0035, 0.9965], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000

tensor([0., 1.]) tensor([1.2513e-04, 9.9987e-01], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([0.3275, 0.6725], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 3.3164e-09], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([0.0387, 0.9613], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([1.0410e-06, 1.0000e+00], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([2.3881e-07, 1.0000e+00], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([2.0575e-05, 9.9998e-01], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([2.1218e-05, 9.9998e-01], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.1001e-04, 9.9989e-01], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 5.7914e-09], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([2.4502e-09, 1.0000e+00], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 4.6146e-07], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([6.2995e-05, 9.9994e-01], grad_fn=<UnbindBackward0>)
tensor([0., 1.]

tensor([1., 0.]) tensor([1.0000e+00, 8.1982e-07], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([8.6888e-04, 9.9913e-01], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([5.0469e-04, 9.9950e-01], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([2.8225e-06, 1.0000e+00], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([7.3941e-07, 1.0000e+00], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 1.0246e-06], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 1.4937e-08], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([1.0511e-06, 1.0000e+00], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([9.9994e-01, 5.6029e-05], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 2.3423e-08], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([0.4761, 0.5239], grad_fn=<UnbindBackward0>)
tensor([0., 1.]) tensor([0.0035, 0.9965], grad_fn=<UnbindBackward0>)
tensor([1., 0.]) tensor([1.0000e+00, 1.3646e-09], grad_fn=<UnbindBackward0>)
tensor([0., 1.]

In [16]:
# Loading input dataframe,
df_input = pd.DataFrame(data = pd.read_csv("model-input.csv"))

# Correcting datatypes,
df_input = df_input.astype("string")
df_input["AGE"] = df_input["AGE"].astype(np.float32)
df_input

,USMR,MEDICAL_UNIT,SEX,PATIENT_TYPE,PNEUMONIA,AGE,PREGNANT,DIABETES,COPD,ASTHMA,INMSUPR,HYPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,CLASSIFICATION
0,No,3,Male,Hospital,No,21.0,No,No,No,No,No,No,No,No,No,No,No,Mild Case
1,No,3,Male,Hospital,No,19.0,No,No,No,No,No,No,No,No,Yes,No,No,Mild Case
2,No,3,Male,Hospital,No,48.0,No,No,No,No,No,Yes,No,No,Yes,No,No,Mild Case
3,No,3,Female,Hospital,No,47.0,No,No,No,No,No,No,No,No,Yes,No,No,Mild Case
4,No,1,Male,Hospital,No,24.0,No,No,No,No,No,No,No,No,No,No,No,Serious Case
5,No,1,Male,Hospital,No,30.0,No,No,No,No,No,No,No,No,No,No,No,Serious Case
6,No,1,Male,Home,No,55.0,No,No,No,No,No,No,No,No,No,No,No,Serious Case
7,No,1,Female,Home,No,48.0,No,Yes,No,No,No,No,No,No,No,No,No,Serious Case
8,No,1,Female,Home,No,23.0,No,No,No,No,No,No,No,No,No,No,No,Serious Case
9,No,1,Female,Hospital,Yes,80.0,No,No,No,No,No,Yes,No,No,No,No,No,Serious Case


In [17]:
columns_encoded = []
for dict_key in test_dataset.features:

    if len(test_dataset.features[dict_key]) != 0:
        for unique_value in test_dataset.features[dict_key]:
            columns_encoded.append(dict_key + "_" + unique_value)
    else:
        columns_encoded.append(dict_key)

df_encoded = pd.DataFrame(columns = columns_encoded)

rows_params = []
for index, row in df_input.iterrows():
    row_param = []
    for dict_key in test_dataset.features:
        if type(row[dict_key]) is str:
            row_param.append(dict_key + "_" + row[dict_key])
        else:
            row_param.append((dict_key, row[dict_key]))
    rows_params.append(row_param)

encoded_rows = []
for row_param in rows_params:
    encoded_row = {}
    for param in row_param:
        if type(param) is str:
            for column in df_encoded:
                if param == column:
                    encoded_row[column] = 1
        else:
            for column in df_encoded:
                if param[0] == column:
                    encoded_row[column] = param[1]
    encoded_rows.append(encoded_row)

for encoded_row in encoded_rows:
    df_encoded.loc[len(df_encoded)] = encoded_row
df_encoded = df_encoded.fillna(0)
df_encoded = df_encoded.astype("float32")
df_encoded

,USMR_No,USMR_Yes,MEDICAL_UNIT_1,MEDICAL_UNIT_2,MEDICAL_UNIT_3,MEDICAL_UNIT_4,MEDICAL_UNIT_5,MEDICAL_UNIT_6,MEDICAL_UNIT_7,MEDICAL_UNIT_8,...,OBESITY_No,OBESITY_Yes,RENAL_CHRONIC_No,RENAL_CHRONIC_Yes,TOBACCO_No,TOBACCO_Yes,CLASSIFICATION_Serious Case,CLASSIFICATION_Negative Case,CLASSIFICATION_Mild Case,CLASSIFICATION_Moderate Case
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
5,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
6,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
7,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
8,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
9,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [13]:
X = df_encoded.to_numpy()
X = torch.tensor(X, dtype = torch.float32)
Y = model(X).detach().numpy()

"""df_Y = pd.DataFrame(columns = ["DESCEASED_No", "DESCEASED_Yes"])
for probabilities in Y:
    df_Y.loc[len(df_Y)] = probabilities"""

Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [21]:
Y

array([[7.1885004e-19, 1.0000000e+00],
       [6.4799470e-17, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [9.5165138e-23, 1.0000000e+00],
       [6.2954584e-30, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [1.7147255e-23, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [5.9582317e-32, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [4.3274513e-27, 1.0000000e+00],
       [3.6335669e-42, 1.0000000e+00],
       [9.2470344e-25, 1.0000000e+00],
       [2.3794577e-34, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [8.8842323e-43, 1.0000000e+00],
       [2.3609118e-28, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 1.